In [1]:
!pip install jsonschema transformers accelerate sentencepiece bitsandbytes tiktoken -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 32.3 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
# Add these tokens in Kaggle's "Secrets" tab (Add-ons -> Secrets)
login(token=user_secrets.get_secret("HF_TOKEN"))

In [3]:
"""
==============================================================
Nyaya-LLM  —  Phase 2 Dataset Augmentation (Kaggle / P100)
==============================================================
Augments all three splits (train, val, test) using a
HuggingFace model loaded directly on the P100 GPU.

Long sections (>3000 chars) are chunked at subsection
boundaries so every part of the statute gets covered.
==============================================================
"""

import json
import re
import os
import time
import torch
import shutil
import gc
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from jsonschema import validate, ValidationError

# ==============================
# ⚙️  CONFIG
# ==============================
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

# ── Paths ───────────────────────────────────────────────────
INPUT_DIR  = "/kaggle/input/datasets/shreyashgaurgla/nyaya-llm-dataset"
OUTPUT_DIR = "/kaggle/working/augmented"
WIP_DATASET_PATH = "/kaggle/input/datasets/shreyashgaurgla/nyaya-augmented-wip" # For Run 2+

SPLITS = {
    "train": "all_acts_train.jsonl"
}

# ── Processing Limits ───────────────────────────────────────
CHUNK_SIZE  = 3000    
RETRY_LIMIT = 3       
MAX_RUNTIME_HOURS = 11.85

# ==============================
# 📦  JSON SCHEMA
# ==============================
AUGMENT_SCHEMA = {
    "type": "object",
    "properties": {
        "instruction": {"type": "string"},
        "input":       {"type": "string"},
        "output":      {"type": "string"}
    },
    "required": ["instruction", "input", "output"]
}

# ==============================
# 💬  PROMPT (CHAT TEMPLATE)
# ==============================
SYSTEM_PROMPT = """You are a strict Indian legal dataset generator.

RULES:
1. Only use the provided statute text. Do NOT add outside legal knowledge.
2. Do NOT invent punishments, sections, or acts not mentioned in the text.
3. Output STRICT JSON only. No markdown, no explanation, no preamble.
4. Return exactly ONE valid JSON object.
5. The instruction must be a realistic question a layperson or law student would ask.
6. The output must apply or explain the statute — not just quote it verbatim."""

def build_messages(original_instruction: str, chunk: str) -> list:
    user_msg = f"""ORIGINAL INSTRUCTION:
{original_instruction}

STATUTE TEXT:
{chunk}

---
Generate a new, enriched training example that:
- Asks a reasoning or application question about this statute text
- Answers it using ONLY the information in the statute text above
- Does NOT reference sections or provisions outside the text above

Respond ONLY with this JSON format, nothing else:
{{
  "instruction": "...",
  "input": "",
  "output": "..."
}}"""

    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_msg}
    ]

# ==============================
# ✂️  CHUNKER
# ==============================
def chunk_text(text: str, max_chars: int = CHUNK_SIZE) -> list:
    if len(text) <= max_chars:
        return [text]

    parts   = text.split("\n\n")
    chunks  = []
    current = ""

    for part in parts:
        if len(part) > max_chars:
            if current:
                chunks.append(current.strip())
                current = ""
            sentences = re.split(r'(?<=[.;])\s+', part)
            for sentence in sentences:
                if len(current) + len(sentence) <= max_chars:
                    current += " " + sentence
                else:
                    if current:
                        chunks.append(current.strip())
                    current = sentence
        elif len(current) + len(part) + 2 <= max_chars:
            current += "\n\n" + part
        else:
            if current:
                chunks.append(current.strip())
            current = part

    if current:
        chunks.append(current.strip())

    return [c for c in chunks if len(c) >= 50]

# ==============================
# 🤖  MODEL LOADING
# ==============================
def load_model():
    print(f"Loading {MODEL_NAME} in 4-bit on GPU...")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16, 
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True
    )
    model.eval()
    print("Model loaded.\n")
    return model, tokenizer

# ==============================
# 🔁  GENERATION (WITH OOM KILLER)
# ==============================
def generate(model, tokenizer, messages: list) -> str:
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=2048
    ).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1024,
            temperature=0.2,
            do_sample=True,
            top_p=0.8,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )

    new_tokens = outputs[0][inputs["input_ids"].shape[1]:]
    result = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    
    del inputs
    del outputs
    del new_tokens
    torch.cuda.empty_cache()
    gc.collect()
    
    return result

# ==============================
# 🔍  JSON EXTRACTION
# ==============================
def extract_json(text: str):
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if not match:
        return None
    try:
        return json.loads(match.group())
    except Exception:
        return None

# ==============================
# 🛡️  HALLUCINATION GUARD (FIXED)
# ==============================
def hallucination_guard(original_text: str, generated_output: str) -> bool:
    out_lower  = generated_output.lower()
    orig_lower = original_text.lower()

    # Lexical overlap only — remove the strict act/section requirement
    orig_words  = set(orig_lower.split())
    gen_words   = set(out_lower.split())
    overlap     = len(orig_words & gen_words)

    # Just needs reasonable overlap with source text
    return overlap > 10

# ==============================
# 💾  CHECKPOINT HELPER
# ==============================
def load_processed_keys(output_path: str) -> set:
    processed = set()
    if os.path.exists(output_path):
        with open(output_path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    obj = json.loads(line)
                    key = (obj.get("source_instruction", ""), obj.get("chunk_index", 0))
                    processed.add(key)
                except Exception:
                    pass
    return processed

# ==============================
# 🚀  MAIN
# ==============================
def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # --- RUN 2 RESTORATION LOGIC ---
    if os.path.exists(WIP_DATASET_PATH):
        print("Restoring previous progress from WIP dataset...")
        for filename in os.listdir(WIP_DATASET_PATH):
            if filename.endswith(".jsonl"):
                src = os.path.join(WIP_DATASET_PATH, filename)
                dst = os.path.join(OUTPUT_DIR, filename)
                shutil.copy2(src, dst)
                print(f"✅ Restored: {filename}")
        print("Ready to resume!\n")
    # -------------------------------
    
    START_TIME = time.time()
    MAX_RUNTIME_SEC = MAX_RUNTIME_HOURS * 3600 

    model, tokenizer = load_model()

    total_written = 0
    total_skipped = 0

    for split_name, filename in SPLITS.items():
        input_path  = os.path.join(INPUT_DIR, filename)
        output_path = os.path.join(OUTPUT_DIR, f"all_acts_{split_name}_augmented.jsonl")

        if not os.path.exists(input_path):
            print(f"⚠️  {input_path} not found — skipping {split_name}")
            continue

        with open(input_path, "r", encoding="utf-8") as f:
            data = [json.loads(line) for line in f]

        processed_keys = load_processed_keys(output_path)
        total_chunks = sum(len(chunk_text(d["output"])) for d in data)

        print(f"\n{'='*60}")
        print(f"📂  Split         : {split_name.upper()}")
        print(f"    Samples       : {len(data)}")
        print(f"    Total chunks  : {total_chunks}")
        print(f"    Already done  : {len(processed_keys)}")
        print(f"    Remaining     : {total_chunks - len(processed_keys)}")
        print(f"{'='*60}\n")

        split_written = 0
        split_skipped = 0

        pbar = tqdm(data, desc=f"Augmenting {split_name}")
        with open(output_path, "a", encoding="utf-8") as outfile:
            for example in pbar:
                instruction = example["instruction"]
                chunks      = chunk_text(example["output"])

                for chunk_idx, chunk in enumerate(chunks):
                    
                    if time.time() - START_TIME > MAX_RUNTIME_SEC:
                        print(f"\n⚠️ REACHED {MAX_RUNTIME_HOURS} HOURS. GRACEFULLY STOPPING.")
                        print(f"📊 Stats this run -> Written: {split_written} | Skipped: {split_skipped}")
                        print(f"💾 Files safely saved to: {OUTPUT_DIR}")
                        return 
                    
                    key = (instruction, chunk_idx)

                    if key in processed_keys:
                        continue

                    retries = 0
                    success = False

                    while retries < RETRY_LIMIT and not success:
                        try:
                            messages = build_messages(instruction, chunk)
                            raw      = generate(model, tokenizer, messages)
                            parsed   = extract_json(raw)

                            if parsed is None:
                                raise ValueError("No JSON found")

                            validate(instance=parsed, schema=AUGMENT_SCHEMA)

                            if not hallucination_guard(chunk, parsed["output"]):
                                raise ValueError("Hallucination guard failed")

                            parsed["source_instruction"] = instruction
                            parsed["chunk_index"]        = chunk_idx
                            parsed["total_chunks"]       = len(chunks)
                            parsed["split"]              = split_name

                            outfile.write(json.dumps(parsed, ensure_ascii=False) + "\n")
                            outfile.flush()
                            success       = True
                            split_written += 1

                        except (ValidationError, ValueError):
                            retries += 1
                            time.sleep(0.2)
                        except Exception as e:
                            print(f"\n❌ Unexpected error: {e}")
                            retries += 1
                            time.sleep(1)

                    if not success:
                        split_skipped += 1
                        
                    # LIVE TRACKER: Updates the progress bar with real-time stats!
                    pbar.set_postfix({"Written": split_written, "Skipped": split_skipped})

        total_written += split_written
        total_skipped += split_skipped

        print(f"\n✅  {split_name} done")
        print(f"    Written : {split_written}")
        print(f"    Skipped : {split_skipped}")
        print(f"    Saved   : {output_path}")

    print(f"\n{'='*60}")
    print(f"🏁  AUGMENTATION COMPLETE")
    print(f"    Total written : {total_written}")
    print(f"    Total skipped : {total_skipped}")
    print(f"    Output dir    : {OUTPUT_DIR}")
    print(f"{'='*60}")

if __name__ == "__main__":
    main()

Restoring previous progress from WIP dataset...
✅ Restored: all_acts_train_augmented.jsonl
Ready to resume!

Loading Qwen/Qwen2.5-7B-Instruct in 4-bit on GPU...


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Model loaded.


📂  Split         : TRAIN
    Samples       : 7752
    Total chunks  : 7935
    Already done  : 4754
    Remaining     : 3181




Augmenting train:  99%|█████████▉| 7702/7752 [11:49:46<04:36,  5.53s/it, Written=992, Skipped=2134]


⚠️ REACHED 11.85 HOURS. GRACEFULLY STOPPING.
📊 Stats this run -> Written: 992 | Skipped: 2134
💾 Files safely saved to: /kaggle/working/augmented
